# SVOD Subscribers – US Market Overview

This report summarizes the key findings from an exploratory analysis of SVOD subscriber data in the United States. The analysis was performed entirely in Python, based on a cleaned dataset derived from the original Excel source.

We focus on market size, company growth, and volatility across time.


---

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

BASE_DIR = Path("D:/GitHub/DataProjects/Dataxis").resolve()
DATA_PROCESSED_DIR = BASE_DIR / "data" / "processed"
REPORTS_DIR = BASE_DIR / "reports"
REPORTS_DIR.mkdir(exist_ok=True)

ts = pd.read_csv(DATA_PROCESSED_DIR / "timeseries_metrics.csv", parse_dates=["fact_date"])
summary = pd.read_csv(DATA_PROCESSED_DIR / "company_summary.csv")

In [ ]:
top_subs = summary.loc[summary["subs_final"].idxmax()]
top_growth_pct = summary.loc[summary["total_growth_pct"].idxmax()]
top_growth_abs = summary.loc[summary["total_growth_abs"].idxmax()]
top_vol = summary.loc[summary["volatility_pct"].idxmax()]

results_text = (
    f"- Largest subscriber base: {top_subs.Company} ({top_subs.subs_final:,.0f} subscribers)\n"
    f"- Highest total percentage growth: {top_growth_pct.Company} ({top_growth_pct.total_growth_pct:.1f}%)\n"
    f"- Highest absolute growth: {top_growth_abs.Company} (+{top_growth_abs.total_growth_abs:,.0f} subscribers)\n"
    f"- Most volatile growth: {top_vol.Company} (std = {top_vol.volatility_pct:.1f} pp)"
)
results_text


'- Largest subscriber base: **Amazon Prime Video** (98,820,754 subscribers)\n- Highest total percentage growth: **Vix Premium** (3247.7%)\n- Highest absolute growth: **Paramount+** (+32,151,000 subscribers)\n- Most volatile growth: **Dove Channel** (std = 92.3 pp)'

In [6]:
# Line chart: top 3 by latest subscribers
top3 = summary.sort_values("subs_final", ascending=False).head(3)["Company"].tolist()
ts_top = ts[ts.Company.isin(top3)]

# Bar chart: top 8 by total growth %
growth_top = summary.sort_values("total_growth_pct", ascending=False).head(8)


In [9]:
fig = plt.figure(figsize=(8.27, 11.69))

# ---------------- HEADER ----------------
fig.suptitle("SVOD Subscribers – US Market Performance (One-Page Summary)",
             fontsize=14, weight="bold", y=0.97)

# ---------------- WHAT WAS ANALYZED ----------------
fig.text(
    0.07, 0.90, "What was analyzed:", fontsize=11, weight="bold"
)

fig.text(
    0.07, 0.865,
    "We computed nine metrics to assess market size, growth dynamics, and stability among US SVOD companies:\n"
    "1. Quarter-over-quarter absolute change (QoQ_abs)\n"
    "2. Quarter-over-quarter percentage change (QoQ_pct)\n"
    "3. Total absolute growth over the full period\n"
    "4. Total percentage growth\n"
    "5. CAGR based on first and last periods\n"
    "6. Quarterly rankings by subscribers\n"
    "7. Rankings by growth (absolute & %)\n"
    "8. Decline and zero-growth flags\n"
    "9. Volatility of quarterly % growth (std)",
    fontsize=8.6, wrap=True
)

# ---------------- WHY IT MATTERS ----------------
fig.text(0.07, 0.73, "Why these metrics matter:", fontsize=11, weight="bold")

fig.text(
    0.07, 0.705,
    "These metrics highlight not only which companies are largest, but which are accelerating, "
    "which are losing momentum, and which face unstable or inconsistent growth patterns. "
    "This combination provides a full performance profile rather than a single static measure.",
    fontsize=8.6, wrap=True
)

# ---------------- KEY RESULTS ----------------
fig.text(0.07, 0.62, "Key findings:", fontsize=11, weight="bold")
fig.text(0.07, 0.56, results_text, fontsize=9, wrap=True)

# ---------------- LINE CHART ----------------
ax1 = fig.add_axes([0.10, 0.33, 0.80, 0.20])
for name, g in ts_top.groupby("Company"):
    ax1.plot(g.fact_date, g.subscribers, marker="o", label=name)
ax1.set_title("Subscribers Over Time (Top 3 Companies)", fontsize=10)
ax1.tick_params(axis="x", rotation=45)
ax1.legend(fontsize=8)

# ---------------- BAR CHART ----------------
ax2 = fig.add_axes([0.10, 0.07, 0.80, 0.20])
ax2.bar(growth_top.Company, growth_top.total_growth_pct)
ax2.set_title("Total % Growth – Top 8 Companies", fontsize=10)
ax2.tick_params(axis="x", rotation=45, labelsize=8)
ax2.set_ylabel("Growth (%)")

# ---------------- EXPORT ----------------
pdf_path = REPORTS_DIR / "SVOD_one_page_full_analysis.pdf"
fig.savefig(pdf_path, bbox_inches="tight")
plt.close(fig)

pdf_path


WindowsPath('D:/GitHub/DataProjects/Dataxis/reports/SVOD_one_page_full_analysis.pdf')